# 登录示例

In [51]:
import itertools
import random
import requests
from pprint import pprint
import time
import random
import json
import duckdb
from datetime import  datetime
from uuid import uuid4

session = requests.session()
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
}
params = {
    "user_id": "fb417c920bb142eabd6016a3520663b3",
    "user_key": "a5737c292dab487a95e6cc31cd8c4d0b",
    "proj_id": "proj_9c2cabfc56bf40d0",
}


In [52]:

def analysis_function(params_list):
    data_list = []
    for i in range(len(params_list)):
        c_request_id = str(uuid4())
        c_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%S+0000")
        c_timestamp = int(time.time())
        json_data = params_list[i]
        res = session.post(
            "https://brdeapi.geetest.com/geerule/run",
            params=params,
            headers=headers,
            json=json_data,
        )
        if res.status_code == 200:
            res_dict = res.json()
            # data_list.append(res_dict['data']['result']['ip_user_ucnt'])
            params_list[i]["date"] = c_date
            params_list[i]["timestamp"] = c_timestamp
            params_list[i]["request_id"] = c_request_id
            res_dict["request_id"] = c_request_id
            data_list.append(res_dict)
            print(res_dict)
            time.sleep(1)
    with open("./data_input.json", "w") as file:
        for item in params_list:
            file.write(json.dumps(item, ensure_ascii=False) + "\n")
            

    with open("./data_output.json", "w") as file:
        for item in data_list:
            file.write(json.dumps(item, ensure_ascii=False) + "\n")


# 单个ip多次访问
params_list1 = [
    {"ip": "1.15.241.228", "user": "15827325750", "scene": "login"},
    {"ip": "1.15.241.228", "user": "15827325750", "scene": "login"},
    {"ip": "1.15.241.228", "user": "15827325750", "scene": "login"},
    {"ip": "1.15.241.228", "user": "15827325750", "scene": "login"},
    {"ip": "1.15.241.228", "user": "15827325750", "scene": "login"},
    {"ip": "1.15.241.228", "user": "15827325750", "scene": "login"},
]

# 单个ip多个用户多次访问（一机器多号）
params_list2 = [
    {"ip": "59.174.225.135", "user": "15827325751", "scene": "login"},
    {"ip": "59.174.225.135", "user": "15827325751", "scene": "login"},
    {"ip": "59.174.225.135", "user": "15827325752", "scene": "login"},
    {"ip": "59.174.225.135", "user": "15827325752", "scene": "login"},
    {"ip": "59.174.225.135", "user": "15827325753", "scene": "login"},
    {"ip": "59.174.225.135", "user": "15827325753", "scene": "login"},
]

# 单个账号用多个ip登录（一号多机）
params_list3 = [
    {"ip": "101.126.32.237", "user": "15827325755", "scene": "login"},
    {"ip": "101.126.32.237", "user": "15827325755", "scene": "login"},
    {"ip": "147.185.132.21", "user": "15827325755", "scene": "login"},
    {"ip": "147.185.132.21", "user": "15827325755", "scene": "login"},
    {"ip": "139.9.223.116", "user": "15827325755", "scene": "login"},
    {"ip": "139.9.223.116", "user": "15827325755", "scene": "login"},
]
analysis_function(params_list1 + params_list2 + params_list3)

{'status': 0, 'data': {'rule_id': 'rule_e07bea135645464d', 'rule_name': '登录', 'performance': '7.320935ms', 'result': [], 'trace_id': '1efbce6710b56fae940652c60cfa2bf4'}, 'message': 'version 1.0', 'request_id': '34ce3ae3-2d9e-4f80-8926-d97ea9a1683d'}
{'status': 0, 'data': {'rule_id': 'rule_e07bea135645464d', 'rule_name': '登录', 'performance': '5.829381ms', 'result': [], 'trace_id': '1efbce671af86056b0d54ef2cdca9402'}, 'message': 'version 1.0', 'request_id': 'e313ef0c-934b-481c-ab3f-e37f908010d0'}
{'status': 0, 'data': {'rule_id': 'rule_e07bea135645464d', 'rule_name': '登录', 'performance': '4.560173ms', 'result': [{'result': '单ip访问次数超过2次'}], 'trace_id': '1efbce6725406f96844b7e0e653806f8'}, 'message': 'version 1.0', 'request_id': '64887486-1142-4e77-9697-b4b746461f4a'}
{'status': 0, 'data': {'rule_id': 'rule_e07bea135645464d', 'rule_name': '登录', 'performance': '5.434127ms', 'result': [{'result': '单ip访问次数超过2次'}], 'trace_id': '1efbce672f7d6c88bfc0f2fd0e4933e0'}, 'message': 'version 1.0', 'req

In [79]:
%load_ext sql
%sql duckdb:///:memory: --alias data_alys_login
%config SqlMagic.displaylimit = None

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Connecting to 'data_alys_login'

displaylimit: Value None will be treated as 0 (no limit)

In [82]:
%sql  CREATE TABLE regs as  SELECT * FROM read_json('data_input.json', columns = {date:'TIMESTAMPTZ', timestamp: 'INTEGER', ip: 'VARCHAR', user: 'VARCHAR'});

Running query in 'data_alys_login'

Count


In [83]:
%sql  CREATE TABLE regs_input as  SELECT * FROM read_json('data_input.json', columns = {date:'TIMESTAMPTZ', timestamp: 'INTEGER', ip: 'VARCHAR', user: 'VARCHAR', request_id:'VARCHAR'});


Running query in 'data_alys_login'

RuntimeError: (duckdb.duckdb.CatalogException) Catalog Error: Table with name "regs_input" already exists!
[SQL: CREATE TABLE regs_input as SELECT * FROM read_json('data_input.json', columns = {date:'TIMESTAMPTZ', timestamp: 'INTEGER' , ip: 'VARCHAR' , user: 'VARCHAR' , request_id:'VARCHAR'});]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [84]:
%%sql SELECT 
    date,
    timestamp,
    ip,
    user,
    COUNT(*) OVER gpv_rate AS gpv_l1m,
    COUNT(*) OVER pv_rate AS pv_l1m,
    COUNT(distinct (user)) OVER uv_rate AS uv_l1m,
    timestamp -lag(timestamp, 1) OVER gidle_count AS gidle,
    timestamp -lag(timestamp, 1) OVER vidle_count AS vidle 
FROM regs 
    WINDOW 
        gpv_rate AS (PARTITION BY ip ORDER BY date RANGE BETWEEN INTERVAL '1 minute' PRECEDING AND CURRENT ROW),
        pv_rate AS (PARTITION BY (ip, user) ORDER BY date RANGE BETWEEN INTERVAL '1 minute' PRECEDING AND CURRENT ROW),
        uv_rate AS (PARTITION BY ip ORDER BY date RANGE BETWEEN INTERVAL '1 minute' PRECEDING AND CURRENT ROW),
        gidle_count AS (PARTITION BY ip ORDER BY date RANGE BETWEEN INTERVAL '1 minute' PRECEDING AND CURRENT ROW),
        vidle_count AS (PARTITION BY (ip, user) ORDER BY date RANGE BETWEEN INTERVAL '1 minute' PRECEDING AND CURRENT ROW) 
ORDER BY date;

Running query in 'data_alys_login'

date,timestamp,ip,user,gpv_l1m,pv_l1m,uv_l1m,gidle,vidle
2024-12-18 02:18:59+00:00,1734488339,1.15.241.228,15827325750,1,1,1,None,None
2024-12-18 02:19:00+00:00,1734488340,1.15.241.228,15827325750,2,2,1,1,1
2024-12-18 02:19:01+00:00,1734488341,1.15.241.228,15827325750,3,3,1,1,1
2024-12-18 02:19:02+00:00,1734488342,1.15.241.228,15827325750,4,4,1,1,1
2024-12-18 02:19:03+00:00,1734488343,1.15.241.228,15827325750,5,5,1,1,1
2024-12-18 02:19:05+00:00,1734488345,1.15.241.228,15827325750,6,6,1,2,2
2024-12-18 02:19:06+00:00,1734488346,59.174.225.135,15827325751,1,1,1,None,None
2024-12-18 02:19:07+00:00,1734488347,59.174.225.135,15827325751,2,2,1,1,1
2024-12-18 02:19:08+00:00,1734488348,59.174.225.135,15827325752,3,1,2,1,None
2024-12-18 02:19:09+00:00,1734488349,59.174.225.135,15827325752,4,2,2,1,1


# 串连出规则引擎返回的结果

In [85]:
%sql  CREATE TABLE regs_input as  SELECT * FROM read_json('data_input.json', columns = {date:'TIMESTAMPTZ', timestamp: 'INTEGER', ip: 'VARCHAR', user: 'VARCHAR', request_id:'VARCHAR'});


Running query in 'data_alys_login'

RuntimeError: (duckdb.duckdb.CatalogException) Catalog Error: Table with name "regs_input" already exists!
[SQL: CREATE TABLE regs_input as SELECT * FROM read_json('data_input.json', columns = {date:'TIMESTAMPTZ', timestamp: 'INTEGER' , ip: 'VARCHAR' , user: 'VARCHAR' , request_id:'VARCHAR'});]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [58]:
%sql CREATE TABLE regs_output as  SELECT data->>'result' as result,request_id as request_id,from read_json("./data_output.json");

Running query in 'data_alys_login'

Count


In [59]:
%sql select *  from  regs_input;

Running query in 'data_alys_login'

date,timestamp,ip,user,request_id
2024-12-18 02:18:59+00:00,1734488339,1.15.241.228,15827325750,34ce3ae3-2d9e-4f80-8926-d97ea9a1683d
2024-12-18 02:19:00+00:00,1734488340,1.15.241.228,15827325750,e313ef0c-934b-481c-ab3f-e37f908010d0
2024-12-18 02:19:01+00:00,1734488341,1.15.241.228,15827325750,64887486-1142-4e77-9697-b4b746461f4a
2024-12-18 02:19:02+00:00,1734488342,1.15.241.228,15827325750,b51abf7f-243c-4cf3-a1d9-cb5f7dceb7f6
2024-12-18 02:19:03+00:00,1734488343,1.15.241.228,15827325750,d88a5b72-2c55-4d3a-920a-700bd2b163d0
2024-12-18 02:19:05+00:00,1734488345,1.15.241.228,15827325750,7d58035f-d584-4123-84f5-0c20d4894295
2024-12-18 02:19:06+00:00,1734488346,59.174.225.135,15827325751,e1161c78-fcea-4f24-ab17-a42bc72b20af
2024-12-18 02:19:07+00:00,1734488347,59.174.225.135,15827325751,d4b259c3-3021-4f94-bb57-88c98807da69
2024-12-18 02:19:08+00:00,1734488348,59.174.225.135,15827325752,e6b45825-95d2-403b-9b6f-7401ecd9f164
2024-12-18 02:19:09+00:00,1734488349,59.174.225.135,15827325752,2fb5f551-3f3a-4542-85ba-7488337c10b2


In [60]:
%sql select request_id,result  from  regs_output;

Running query in 'data_alys_login'

request_id,result
34ce3ae3-2d9e-4f80-8926-d97ea9a1683d,[]
e313ef0c-934b-481c-ab3f-e37f908010d0,[]
64887486-1142-4e77-9697-b4b746461f4a,"[{""result"":""单ip访问次数超过2次""}]"
b51abf7f-243c-4cf3-a1d9-cb5f7dceb7f6,"[{""result"":""单ip访问次数超过2次""}]"
d88a5b72-2c55-4d3a-920a-700bd2b163d0,"[{""result"":""单ip访问次数超过2次""}]"
7d58035f-d584-4123-84f5-0c20d4894295,"[{""result"":""单ip访问次数超过2次""}]"
e1161c78-fcea-4f24-ab17-a42bc72b20af,[]
d4b259c3-3021-4f94-bb57-88c98807da69,[]
e6b45825-95d2-403b-9b6f-7401ecd9f164,"[{""result"":""单ip访问次数超过2次""}]"
2fb5f551-3f3a-4542-85ba-7488337c10b2,"[{""result"":""单ip访问次数超过2次""}]"


In [61]:
%sql CREATE TABLE regs_all as  select regs_input.date,regs_input.timestamp,regs_input.ip,regs_input.user, regs_output.result from  regs_input  left join regs_output on regs_input.request_id = regs_output.request_id;

Running query in 'data_alys_login'

Count


In [78]:
%sql select *  from  regs_all;

Running query in 'data_alys_login'

date,timestamp,ip,user,result
2024-12-18 02:18:59+00:00,1734488339,1.15.241.228,15827325750,[]
2024-12-18 02:19:00+00:00,1734488340,1.15.241.228,15827325750,[]
2024-12-18 02:19:01+00:00,1734488341,1.15.241.228,15827325750,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:02+00:00,1734488342,1.15.241.228,15827325750,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:03+00:00,1734488343,1.15.241.228,15827325750,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:05+00:00,1734488345,1.15.241.228,15827325750,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:06+00:00,1734488346,59.174.225.135,15827325751,[]
2024-12-18 02:19:07+00:00,1734488347,59.174.225.135,15827325751,[]
2024-12-18 02:19:08+00:00,1734488348,59.174.225.135,15827325752,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:09+00:00,1734488349,59.174.225.135,15827325752,"[{""result"":""单ip访问次数超过2次""}]"


In [66]:
# -- 单ip 访问次数 累计次数超过2次 pv
%sql select date,ip as g, user as v, count(*) OVER (PARTITION BY g ORDER BY date RANGE BETWEEN INTERVAL '1 minute' PRECEDING AND CURRENT ROW) as gpv_l1m,result from regs_all order by g,date;

Running query in 'data_alys_login'

date,g,v,gpv_l1m,result
2024-12-18 02:18:59+00:00,1.15.241.228,15827325750,1,[]
2024-12-18 02:19:00+00:00,1.15.241.228,15827325750,2,[]
2024-12-18 02:19:01+00:00,1.15.241.228,15827325750,3,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:02+00:00,1.15.241.228,15827325750,4,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:03+00:00,1.15.241.228,15827325750,5,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:05+00:00,1.15.241.228,15827325750,6,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:12+00:00,101.126.32.237,15827325755,1,[]
2024-12-18 02:19:13+00:00,101.126.32.237,15827325755,2,[]
2024-12-18 02:19:16+00:00,139.9.223.116,15827325755,1,"[{""result"":""单用户多设备累计设备数超过2次""}]"
2024-12-18 02:19:17+00:00,139.9.223.116,15827325755,2,"[{""result"":""单用户多设备累计设备数超过2次""}]"


In [69]:
# -- 单ip 访问次数 user去重超过2次 uv
%sql select date,ip as g, user as v, count(distinct v) OVER (PARTITION BY g ORDER BY date RANGE BETWEEN INTERVAL '1 minute' PRECEDING AND CURRENT ROW) as uv_l1m,result from regs_all order by g,date;

Running query in 'data_alys_login'

date,g,v,uv_l1m,result
2024-12-18 02:18:59+00:00,1.15.241.228,15827325750,1,[]
2024-12-18 02:19:00+00:00,1.15.241.228,15827325750,1,[]
2024-12-18 02:19:01+00:00,1.15.241.228,15827325750,1,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:02+00:00,1.15.241.228,15827325750,1,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:03+00:00,1.15.241.228,15827325750,1,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:05+00:00,1.15.241.228,15827325750,1,"[{""result"":""单ip访问次数超过2次""}]"
2024-12-18 02:19:12+00:00,101.126.32.237,15827325755,1,[]
2024-12-18 02:19:13+00:00,101.126.32.237,15827325755,1,[]
2024-12-18 02:19:16+00:00,139.9.223.116,15827325755,1,"[{""result"":""单用户多设备累计设备数超过2次""}]"
2024-12-18 02:19:17+00:00,139.9.223.116,15827325755,1,"[{""result"":""单用户多设备累计设备数超过2次""}]"


In [48]:
# 单用户多设备累计设备数 不去重 pv 

In [47]:
# 单用户多设备累计设备数超过2次 去重 uv

In [49]:
# 单设备多用户累计用户数超过2次 不去重 pv 

In [50]:
# 单设备多用户累计用户数超过2次 去重 uv